<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Procesamiento de Lenguaje Natural</h1>

<h3>LSTM básico: Haciendo N-gramas</h3>


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/pln/blob/main/labs/RNN/deep-ngram.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

Tomado parcialmente y adaptado de [el repositorio de github](https://github.com/shaundsouza/lstm-textual-ngrams) del trabajo [*LSTM Neural Network for Textual Ngrams* (D'Souza, 2018)](https://www.preprints.org/manuscript/201811.0579/v1)



</center>


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

## Importando datos de alguna obra 

Vamos a descargar un *Corpus*, y pues vamos a usar algo muy famoso, como  puede ser la obra de Shakespiare en inglés, o *El Quijote* en español. Empecemos por *El Quijote*

In [ ]:
!curl -o quijote.txt https://www.gutenberg.org/cache/epub/2000/pg2000.txt

y ahora vamos a descargar el corpus por lineas de texto, eliminando cabecera y final de Gutemberg, quitando espacios, moviendo todas las letras a minúsculas.

In [ ]:
archivo = "quijote.txt"
corpus = []
with open(archivo, 'r', encoding='utf8') as fp:
    guarda = False
    for line in fp.readlines():
      if "*** END OF THE PROJECT GUTENBERG" in line:
        break
      if guarda and len(line) > 2:
        corpus.append(line.strip())
      if "*** START OF THE PROJECT GUTENBERG" in line:
        guarda = True

Veamos como de ve el corpus:

In [ ]:
print(f"Inicio del texto: \n{corpus[:10]}")

In [ ]:
print(f"Fin del texto: \n{corpus[-10:]}")

In [ ]:
print(type(corpus))
print(len(corpus))

## Preprocesando la información 

Para el preprocesamiento, vamos a convertir a tokens cada linea de texto y vamos a agregar pads al mas largo de los textos que se encuentran en el documento (antes de cada salto de linea). Vamos a utilizar la capa de `TextVectorization` que ofrece `Keras`:

In [ ]:
max_tokens = 10_000

indizador = layers.TextVectorization(max_tokens=max_tokens)
indizador.adapt(corpus)

Por lo que tenemos un nuevo vocabulario que podemos ver en orden:

In [ ]:
vocab = indizador.get_vocabulary()
print(f"Tenemos un vocabulario de {len(vocab)} tokens")
print(f"Y aquí están las primeras 20 palabras:\n{vocab[:20]}")

Y ahora vamos a convertir el *corpus* en tensores de tokens simplemente como:

In [ ]:
entradas = indizador(corpus)

print(f"Las entradas en un tensor con un shape de:\n{entradas.shape}")

Como podemos ver tenemos una serie de tokens por linea (o muestra) que se generó del *corpus*. Ahora vamos poniendo la secuencia de salida. 

Como lo que queremos es estimar la próxima palabra, pues no queda mas que usar los mismos tokens de entrada, pero con un corrimiento hacia la izquierda:

In [ ]:
salidas = tf.roll(entradas, shift=-1, axis=1)

print(f"Entrada 1,000: \n{entradas[1_000]}")
print(f"Salida 1,000: \n{salidas[1_000]}")

Como podemos ver, las salidas son iguales a las entradas pero con un adelanto, así ya podemos especificar nuestro modelo de N-gramas.

# Modelo y entrenamiento

El modelo es muy sencillo y podemos discutirlo mucho y modificarlo para probar nuevas cosas.